# Estimación de kilometraje automoviles kaufmann
Este programa toma informacion de ot´s generadas por SAP y procesadas a travez SAS. Las caracteristicas principales son:
* Cada linea representa un vehiculo y su venida o no a servicio
* Si un vehiculo no viene a servicio entonces solo figura una sola vez
* Si un vehiculo viene varias veces a servicio, se duplican las filas tantas veces como sea necesario.

Se utilizan las siguientes variables:

* ‘NRO_CHASSIS’: esta variable es puramente para registrar la predicción y poder hacer un calculo del error a futuro.
* ‘RUT’: Determinar si es persona jurídica o no.
* ‘FAMILIA’
* ‘MODELO’
* ‘MODELO_DESCRIPCION’
* ‘SEXO_CLIENTE’
* ‘PROVINCIA_CLIENTE’
* ‘MONTO_VENTA’
* ‘MONEDA’
* ‘TIPO_CAMBIO’
* ‘FORMA_DE_PAGO’

In [21]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split

## Lectura de datos con el data cleaning listo
Se realizo la siguiente limpieza de datos:
* Generacion de KML_POR_MES = (kilometraje/dias(fecha atencion-fecha de venta))*30
* Dias entre (fecha atencion-fecha de venta) >60
* Valores solo entre p1 < kml_por_mes < p99
* Modelo_descricpion se transforma a variable blue_efficiency (binaria) y modelo de vehiculo (multinomial)
* Monto venta todo el clp
* Se descartan valores de autos usados
* Solo familias NGCC, MID SIZE, SUV o DREAM CARS

In [22]:
mainpath = "/Users/salibaud/Desktop/python/kaufmann/ESTIMADOR DE KML AUTOMOVILES"
filename = "ML_ESTIMACION_KML.xlsx"
fullpath = os.path.join(mainpath, filename)
#data = pd.read_sas(r"\\10.1.1.58\datos\CHILE\IMPORTER_BUSINESS_MANAGEMENT\ml_estimacion_kml.sas7bdat")
xl = pd.ExcelFile(fullpath)
xl
data = xl.parse('Sheet1')
data.head()


,NRO_CHASSIS,PROVINCIA_CLIENTE,FORMA_DE_PAGO,FAMILIA,SEXO_CLIENTE,KML_POR_MES,BLUE_EFFICIENCY,MODELO_VEHICULO,MONTO_VENTA,TIPO_PERS
0,WDD2120651A890108,VALPARAISO,NORMAL,MID SIZE,2,761.297710,0,CLASE E,3.753816e+07,0
1,WDD2120651A890108,VALPARAISO,NORMAL,MID SIZE,2,534.046021,0,CLASE E,3.753816e+07,0
2,WDD2120651A890108,VALPARAISO,NORMAL,MID SIZE,2,365.498339,0,CLASE E,3.753816e+07,0
3,WDD2130421A019733,CAUQUENES,NORMAL,MID SIZE,1,684.586466,0,CLASE E,2.788069e+07,0
4,WDD2130421A019733,CAUQUENES,NORMAL,MID SIZE,1,676.289062,0,CLASE E,2.788069e+07,0


## Inicio del Data Wrangling
* Generacion de variables dummy

In [23]:
def variables_dummy(dataset,variable):
    cat_dummies = pd.get_dummies(dataset[variable],prefix=variable)
    #.iloc[:,1:]
    data_new = dataset.join(cat_dummies)
    dataset = data_new.drop([variable],axis=1)
    return dataset

In [24]:
tabla=variables_dummy(data,"PROVINCIA_CLIENTE")
tabla=variables_dummy(tabla,"MODELO_VEHICULO")
tabla=variables_dummy(tabla,"FAMILIA")
tabla=variables_dummy(tabla,"TIPO_PERS")
tabla=variables_dummy(tabla,"SEXO_CLIENTE")
tabla=variables_dummy(tabla,"FORMA_DE_PAGO")
data.columns

Index(['NRO_CHASSIS', 'PROVINCIA_CLIENTE', 'FORMA_DE_PAGO', 'FAMILIA',
       'SEXO_CLIENTE', 'KML_POR_MES', 'BLUE_EFFICIENCY', 'MODELO_VEHICULO',
       'MONTO_VENTA', 'TIPO_PERS'],
      dtype='object')

In [25]:
y=tabla["KML_POR_MES"]
x=tabla.drop(["KML_POR_MES","NRO_CHASSIS"], axis=1)
x.columns


Index(['BLUE_EFFICIENCY', 'MONTO_VENTA', 'PROVINCIA_CLIENTE_ANTOFAGASTA',
       'PROVINCIA_CLIENTE_ARAUCO', 'PROVINCIA_CLIENTE_ARICA',
       'PROVINCIA_CLIENTE_BIO BIO', 'PROVINCIA_CLIENTE_CACHAPOAL',
       'PROVINCIA_CLIENTE_CALERA DE TANGO', 'PROVINCIA_CLIENTE_CARDENAL CARO',
       'PROVINCIA_CLIENTE_CAUQUENES', 'PROVINCIA_CLIENTE_CAUTIN',
       'PROVINCIA_CLIENTE_CHACABUCO', 'PROVINCIA_CLIENTE_CHAÑARAL',
       'PROVINCIA_CLIENTE_CHILOE', 'PROVINCIA_CLIENTE_CHOAPA',
       'PROVINCIA_CLIENTE_COIHAIQUE', 'PROVINCIA_CLIENTE_COLCHAGUA',
       'PROVINCIA_CLIENTE_COLINA', 'PROVINCIA_CLIENTE_CONCEPCION',
       'PROVINCIA_CLIENTE_COPIAPO', 'PROVINCIA_CLIENTE_CORDILLERA',
       'PROVINCIA_CLIENTE_COYHAIQUE', 'PROVINCIA_CLIENTE_CURICO',
       'PROVINCIA_CLIENTE_EL LOA', 'PROVINCIA_CLIENTE_ELQUI',
       'PROVINCIA_CLIENTE_HUASCO', 'PROVINCIA_CLIENTE_IQUIQUE',
       'PROVINCIA_CLIENTE_LIMARI', 'PROVINCIA_CLIENTE_LINARES',
       'PROVINCIA_CLIENTE_LLANQUIHUE', 'PROVINCIA_CLIENTE_LOS

In [26]:
x_train,x_test,y_train,y_test=train_test_split(x,y)

In [27]:
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import  AdaBoostRegressor
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import metrics 

import warnings
import numpy as np

In [28]:
modelos = [["LM","RIDGE","BRIDGE","TREE","RandomForest","GBOOST","ADABOOST","MLPREGRESOR"],
           [LinearRegression(fit_intercept=True, normalize=True),
            Ridge(),
            BayesianRidge(),
            DecisionTreeRegressor(max_depth=30),
            RandomForestRegressor(),
            GradientBoostingRegressor(),
            AdaBoostRegressor(),
            MLPRegressor(hidden_layer_sizes=(60,),  activation='tanh', solver='lbfgs', alpha=0.00001)]
          ]
warnings.filterwarnings("ignore")

In [29]:
resultados=[]
predicion=[]
error=[]
precision=[]
a=0
for i in modelos[1]:
    ajuste=i.fit(x_train,y_train)
    resultados.append(ajuste)
    predicion.append(ajuste.predict(x_test))
    er = np.sqrt(metrics.mean_squared_error(y_test,ajuste.predict(x_test)))
    error.append(er)
    precision.append(ajuste.score(x_test , y_test))
    print("modelo:",modelos[0][a])
    print("ajuste de:", ajuste.score(x_test , y_test))
    a=a+1

modelo: LM
ajuste de: 0.14510635168979447
modelo: RIDGE
ajuste de: 0.14495536437304135
modelo: BRIDGE
ajuste de: 0.14509262792985367
modelo: TREE
ajuste de: 0.6365874172846447
modelo: RandomForest
ajuste de: 0.6960378145055581
modelo: GBOOST
ajuste de: 0.17179755828168786
modelo: ADABOOST
ajuste de: -0.20739161403772677
modelo: MLPREGRESOR
ajuste de: -1.612805199746603e-05


### De los modelos analizados, buscamos el que tenga mayor presicion

In [30]:
# RMSE
resumen=pd.DataFrame(list(zip(np.array(modelos[0]).T.tolist(),error,precision)))
resumen.columns=["Modelo","Error","Precision"]
#resumen

In [31]:
#busco maxima precision
max_pre=resumen.max(axis=0)["Precision"]
#busco modelo
selec=resumen[resumen["Precision"]==max_pre]

In [32]:
a=selec["Modelo"]
for i,w in zip(modelos[0],modelos[1]):
    for j in selec["Modelo"]:
        if i == j:
            modelo_final = w

In [33]:
#modelo_final

In [14]:
#importancia = pd.DataFrame(list(zip(x_train.columns, modelo_final.feature_importances_)))
#importancia.sort_values(1,ascending=False)

In [34]:
modelo_final.predict(x_test)
x_test.columns

Index(['BLUE_EFFICIENCY', 'MONTO_VENTA', 'PROVINCIA_CLIENTE_ANTOFAGASTA',
       'PROVINCIA_CLIENTE_ARAUCO', 'PROVINCIA_CLIENTE_ARICA',
       'PROVINCIA_CLIENTE_BIO BIO', 'PROVINCIA_CLIENTE_CACHAPOAL',
       'PROVINCIA_CLIENTE_CALERA DE TANGO', 'PROVINCIA_CLIENTE_CARDENAL CARO',
       'PROVINCIA_CLIENTE_CAUQUENES', 'PROVINCIA_CLIENTE_CAUTIN',
       'PROVINCIA_CLIENTE_CHACABUCO', 'PROVINCIA_CLIENTE_CHAÑARAL',
       'PROVINCIA_CLIENTE_CHILOE', 'PROVINCIA_CLIENTE_CHOAPA',
       'PROVINCIA_CLIENTE_COIHAIQUE', 'PROVINCIA_CLIENTE_COLCHAGUA',
       'PROVINCIA_CLIENTE_COLINA', 'PROVINCIA_CLIENTE_CONCEPCION',
       'PROVINCIA_CLIENTE_COPIAPO', 'PROVINCIA_CLIENTE_CORDILLERA',
       'PROVINCIA_CLIENTE_COYHAIQUE', 'PROVINCIA_CLIENTE_CURICO',
       'PROVINCIA_CLIENTE_EL LOA', 'PROVINCIA_CLIENTE_ELQUI',
       'PROVINCIA_CLIENTE_HUASCO', 'PROVINCIA_CLIENTE_IQUIQUE',
       'PROVINCIA_CLIENTE_LIMARI', 'PROVINCIA_CLIENTE_LINARES',
       'PROVINCIA_CLIENTE_LLANQUIHUE', 'PROVINCIA_CLIENTE_LOS

In [16]:
#modelo_final.predict(x_test)

In [17]:
from sklearn.externals import joblib

In [18]:
joblib.dump(modelo_final,'modelo_entrenado.pkl')

['modelo_entrenado.pkl']

In [19]:
#x.columns.values.tolist()

In [20]:
import requests

SERVER='http://api.odd.co'

login = {
    'username': 'kaufmann_api',
    'password': 'BpZgb74qtk',
}
results = requests.post(SERVER + '/clairvoyant/jwt-auth/', json=login)



In [9]:
token = results.json()['token']

JSONDecodeError: Expecting value: line 1 column 1 (char 0)